# Clustering Internals

Understanding the data structures and visualization tools behind time series clustering.

This notebook demonstrates:

- **Data structures**: `Clustering`, `ClusterResult`, and `ClusterStructure`
- **Plot accessor**: Built-in visualizations via `.plot`
- **Data expansion**: Using `expand_data()` to map aggregated data back to original timesteps

!!! note "Prerequisites"
    This notebook assumes familiarity with [08c-clustering](08c-clustering.ipynb).

In [ ]:
from data.generate_example_systems import create_district_heating_system

import flixopt as fx

fx.CONFIG.notebook()

flow_system = create_district_heating_system()
flow_system.connect_and_transform()

## Clustering Metadata

After calling `cluster()`, metadata is stored in `fs.clustering`:

In [ ]:
fs_clustered = flow_system.transform.cluster(
    n_clusters=8,
    cluster_duration='1D',
    time_series_for_high_peaks=['HeatDemand(Q_th)|fixed_relative_profile'],
)

fs_clustered.clustering

The `Clustering` contains:
- **`result`**: A `ClusterResult` with timestep mapping and weights
- **`result.cluster_structure`**: A `ClusterStructure` with cluster assignments

In [ ]:
fs_clustered.clustering.result

In [ ]:
fs_clustered.clustering.result.cluster_structure

## Visualizing Clustering

The `.plot` accessor provides built-in visualizations for understanding clustering results.

In [ ]:
# Compare original vs aggregated data as timeseries
# By default, plots all time-varying variables
fs_clustered.clustering.plot.compare()

In [ ]:
# Use a different approach of visualizing the data using normalize heatmaps
ds = fs_clustered.clustering.plot.compare(data_only=True).data

ds_normalized = (ds - ds.min()) / (ds.max() - ds.min())
ds_normalized.to_array().plotly.imshow(
    x='time',
    animation_frame='representation',
    zmin=0,
    zmax=1,
    color_continuous_scale='viridis',
    title='Normalized Comparison',
)

In [ ]:
# Compare specific variables only
fs_clustered.clustering.plot.compare(variables='HeatDemand(Q_th)|fixed_relative_profile')

In [ ]:
# Duration curves show how well the aggregated data preserves the distribution
fs_clustered.clustering.plot.compare(kind='duration_curve').data

In [ ]:
# View typical period profiles for each cluster
# Each line represents a cluster's representative day
fs_clustered.clustering.plot.clusters(variables='HeatDemand(Q_th)|fixed_relative_profile')

In [ ]:
# Heatmap shows cluster assignments for each original period
fs_clustered.clustering.plot.heatmap()

## Expanding Aggregated Data

The `ClusterResult.expand_data()` method maps aggregated data back to original timesteps.
This is useful for comparing clustering results before optimization:

In [ ]:
# Get original and aggregated data
result = fs_clustered.clustering.result
original = result.original_data['HeatDemand(Q_th)|fixed_relative_profile']
aggregated = result.aggregated_data['HeatDemand(Q_th)|fixed_relative_profile']

# Expand aggregated data back to original timesteps
expanded = result.expand_data(aggregated)

print(f'Original:   {len(original.time)} timesteps')
print(f'Aggregated: {len(aggregated.time)} timesteps')
print(f'Expanded:   {len(expanded.time)} timesteps')

## Summary

| Class | Purpose |
|-------|--------|
| `Clustering` | Stored on `fs.clustering` after `cluster()` |
| `ClusterResult` | Contains timestep mapping, weights, and `expand_data()` method |
| `ClusterStructure` | Maps original periods to clusters |

### Plot Accessor Methods

| Method | Description |
|--------|-------------|
| `plot.compare()` | Compare original vs aggregated data (timeseries) |
| `plot.compare(kind='duration_curve')` | Compare as duration curves |
| `plot.clusters()` | View each cluster's profile |
| `plot.heatmap()` | Visualize cluster assignments |

### Key Parameters

```python
# Compare with options
clustering.plot.compare(
    variables='Demand|profile',       # Single variable, list, or None (all)
    kind='timeseries',                # 'timeseries' or 'duration_curve'
    select={'scenario': 'Base'},      # xarray-style selection
    colors='viridis',                 # Colorscale name, list, or dict
    facet_col='period',               # Facet by period if present
    facet_row='scenario',             # Facet by scenario if present
)

# Heatmap shows cluster assignments (no variable needed)
clustering.plot.heatmap()

# Expand aggregated data to original timesteps
result = clustering.result
expanded = result.expand_data(aggregated_data)
```

## Cluster Weights

Each representative timestep has a weight equal to the number of original periods it represents.
This ensures operational costs scale correctly:

$$\text{Objective} = \sum_{t \in \text{typical}} w_t \cdot c_t$$

The weights sum to the original timestep count:

In [ ]:
print(f'Sum of weights: {fs_clustered.cluster_weight.sum().item():.0f}')
print(f'Original timesteps: {len(flow_system.timesteps)}')

## Solution Expansion

After optimization, `expand()` maps results back to full resolution:

In [ ]:
solver = fx.solvers.HighsSolver(mip_gap=0.01, log_to_console=False)
fs_clustered.optimize(solver)

fs_expanded = fs_clustered.transform.expand()

print(f'Clustered: {len(fs_clustered.timesteps)} timesteps')
print(f'Expanded:  {len(fs_expanded.timesteps)} timesteps')